<a href="https://colab.research.google.com/github/nishokn/ADS-509---Module-4/blob/main/Political_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [ ]:
import sqlite3
import nltk
import random
import numpy as np
import re
from collections import Counter, defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text
for each party and prepare it for use in Naive Bayes.

In [ ]:
convention_data = []
# fill this list up with items that are themselves lists. The
# sublists will have two elements. The first element in the sublist
# should be a string of the cleaned words in the speech. The second
# element should be the party.

# Define the cleaning and tokenizing function
def clean_tokenize(text):
    # Convert text to lowercase and tokenize
    tokens = nltk.word_tokenize(text.lower())
    # Keep only alphabetic tokens
    tokens = [token for token in tokens if token.isalpha()]
    # Return as a single string of tokens
    return ' '.join(tokens)

query_results = convention_cur.execute(
    '''
    SELECT text, party
    FROM conventions
    '''
)

for row in query_results :
    text = row[0]  # the raw text
    party = row[1]  # the associated party
    cleaned_text = clean_tokenize(text)
    convention_data.append([cleaned_text, party])

In [ ]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right.

In [ ]:
random.choices(convention_data,k=5)

[['and the benefits of that response extend far beyond coronavirus will continue to aid many that are just unable to find transportation or way to the doctor for regular checkups this is especially true in rural america i live in a town of about people we do not have buses trains trolleys or uber s available to us in addition the unavailability of services can also hinder treatment for many so increased access to from millions of americans has truly been and we have president trump to thank',
  'Republican'],
 ['that s good well say hello to the folks in utah because they are great people thank you very much congratulations',
  'Republican'],
 ['and then he comes back to me then he goes it was lovely talking to your and then he starts the interview with dana bash it was unbelievable',
  'Democratic'],
 ['what she says for the people inaudible every ounce of who she is she is for us she s for us',
  'Democratic'],
 ['gino is a truck driver from ohio who heard politicians for years make 

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html).

In [ ]:
conv_sent_data = []

for speech, party in convention_data:
    # Tokenize the speech into sentences
    sentences = sent_tokenize(speech)
    # Append each sentence with the associated party to conv_sent_data
    for sentence in sentences:
        conv_sent_data.append([sentence, party])


Again, let's look at some random entries.

In [ ]:
random.choices(conv_sent_data,k=5)

[['chairman coming to you live from the wisconsin center it s time to begin our virtual trip around america our journey begins at the site of a major step forward in our national journey towards justice let s go to alabama',
  'Democratic'],
 ['northern mariana islands', 'Democratic'],
 ['someone who looks like us on a presidential ticket that s crazy',
  'Democratic'],
 ['he understands that leadership means fighting for the people who built this country all of you boyle allred kenyatta kimpson all of us',
  'Democratic'],
 ['i m voting for joe biden because it s on my generation to make sure that we never go back',
  'Democratic']]

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps:

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [ ]:
stop_words = set(stopwords.words('english'))

clean_conv_sent_data = []  # list of tuples (cleaned sentence, party)

for idx, sent_party in enumerate(conv_sent_data):
    sentence, party = sent_party

    tokens = sentence.split()

    # Remove punctuation, non-alphabetic tokens, and stopwords
    cleaned_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]

    # Join the remaining tokens into a cleaned sentence
    cleaned_sentence = ' '.join(cleaned_tokens)

    # Append the cleaned sentence and party to the clean_conv_sent_data list
    clean_conv_sent_data.append((cleaned_sentence, party))

random.choices(clean_conv_sent_data, k=5)


[('difficult times stand closest together tragedy go stronger', 'Democratic'),
 ('nebraska', 'Republican'),
 ('great back andrew thank please', 'Republican'),
 ('official roll call business republican convention conducted today charlotte created short video symbolize excitement president trump across states territories thank watching god bless god bless united states america',
  'Republican'),
 ('thanks voters across country red states blue states fried kimpson nez r',
  'Democratic')]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5.

In [ ]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)

print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2376 as features in the model.


In [ ]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.

       Args:
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word
            in `text` must be in fw in order to be returned. This
            prevents us from considering very rarely occurring words.

       Returns:
            A dictionary with the words in `text` that appear in `fw`.
            Words are only counted once.
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of
            {'quick' : True,
             'fox' :    True}

    """

    # Your code here
    tokens = text.split()
    ret_dict = dict()
    for token in tokens:
        if token in fw:
            ret_dict[token] = True
    return(ret_dict)

In [ ]:
assert(len(feature_words)>0)
assert(conv_features("obama is the president",feature_words)==
       {'obama':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory.

In [ ]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [ ]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [ ]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.434


In [ ]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Based on the results of the classifier, my observations are as follows:

Republican voters seem to be more focused on issues regarding: China, enforcement, destruction, freedoms, support, crime rates, media, beliefs, country pride, national defense, isis, liberals, religion, trade, flag, greatness, etc. The Republican voters are particularly keen on areas regarding trade with China, religion, ensuring personal freedoms, and upholding conservative values. Foreign policy with China seems to be their most pressing issue according to tweets while the amendments are their least pressing issue.

Democrat voters seem to be more focused on issues such as voting rights/voting registration and climate change with voting rights and registration being more of a pressing issue than climate change.

There is not much overlap in terms of rhetoric, with Republican voters' rhetoric being more on the extreme side. Republican voters also have a bias against mainstream media as 'media' is one of the more pressing issues Republicans feel they are facing. Their tweets tend to indicate disdain.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and
is unindexed, so the query takes a minute or two to run on my machine.

In [ ]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [ ]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT
                  cd.candidate,
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle
               AND cd.candidate == tw.candidate
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic')
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

[('Mo Brooks',
  'Republican',
  b'"Brooks Joins Alabama Delegation in Voting Against Flawed Funding Bill" http://t.co/3CwjIWYsNq')]

In [ ]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results:
    party = row[1]
    name = row[0]
    tweet = row[2].decode('utf-8')  # Convert the tweet text to a string
    cleaned_text = clean_tokenize(tweet)
    tweet_data.append([cleaned_text, party])

print(tweet_data[:1])

[['brooks joins alabama delegation in voting against flawed funding bill http', 'Republican']]


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [ ]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [ ]:
def extract_tweet_features(words):
    return {word: True for word in words}

# Create feature sets using the extract_tweet_features function
featuresets = [(extract_tweet_features(tweet), party) for tweet, party in tweet_data]

random.seed(20201014)
random.shuffle(featuresets)

split_index = int(len(featuresets) * 0.7)
train_set = featuresets[:split_index]
test_set = featuresets[split_index:]

# Train the Naive Bayes classifier using the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Classify the sample tweets and compare the results to the actual party labels
for tweet, party in tweet_data_sample:
    tweet_features = extract_tweet_features(tweet)
    estimated_party = classifier.classify(tweet_features)

    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifier says {estimated_party}.")
    print("")


Here's our (cleaned) tweet: the mass shooting in las vegas was a horrific act of violence the victims and their families are in my thoughts and prayers
Actual party is Democratic and our classifier says Democratic.

Here's our (cleaned) tweet: early morning this traveltuesday leaving for dc http
Actual party is Republican and our classifier says Democratic.

Here's our (cleaned) tweet: the only moderates in iraq amp syria are civilians we enemies on both sides of the conflict we should not assist either
Actual party is Republican and our classifier says Democratic.

Here's our (cleaned) tweet: rt natsecaction over national security veterans are demanding answers on the release of the confidential national security
Actual party is Democratic and our classifier says Democratic.

Here's our (cleaned) tweet: buildthatwall now https
Actual party is Republican and our classifier says Democratic.

Here's our (cleaned) tweet: i was glad to attend the and assure everyone i could that majority o

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [ ]:
# dictionary of counts by actual party and estimated party.
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp
    # Now do the same thing as above, but we store the results rather
    # than printing them.

    # get the estimated party
    featuresets = extract_tweet_features(tweet)
    estimated_party = classifier.classify(featuresets)

    results[party][estimated_party] += 1

    if idx > num_to_score :
        break

In [ ]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 1, 'Democratic': 4072}),
             'Democratic': defaultdict(int,
                         {'Republican': 0, 'Democratic': 5929})})

### Reflections

Based on these results, there is 1 Republican tweet correctly identified as Republican with 4072 incorrectly identified as Democrat when it was a Republican tweet.

5929 Democrat tweets were correctly identified as Democrat tweets while 0 Democrat tweets were identified as Republican. This is not very likely and indicates that the code did not work correctly somewhere along the process.